In [64]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

In [78]:
dev = qml.device("default.qubit", wires=5)


def get_H(h, j):
    op = h * qml.PauliX(4)
    for i in range(5-1):
        op -= j * qml.PauliZ(i) @ qml.PauliZ(i+1)
        op += h * qml.PauliX(i)
        
    return op


def my_quantum_function(params, wires):
#     qml.templates.BasicEntanglerLayers([[0]*5], wires=range(5))     
    k = 0
    for i in range(5):
        qml.Hadamard(wires=i)
        qml.RX(params[k], wires=i)
        qml.RY(params[k+1], wires=i)
        qml.RZ(params[k+2], wires=i)
        k += 3

    

In [79]:
H = get_H(h=0.5, j=1)
cost = qml.ExpvalCost(my_quantum_function, H, dev)
opt = qml.GradientDescentOptimizer(stepsize=0.4)
weights = np.random.uniform(0, np.pi, 5*3)
for i in range(20):
    loss_val = cost(weights)
    print(loss_val)
    weights = opt.step(cost, weights)
    

-3.324068590603738
-3.9739029405849924
-4.150653520756314
-4.189302782487551
-4.206705777703146
-4.219875131093094
-4.231972387118484
-4.243728730236398
-4.255274195334969
-4.266579645654436
-4.27757764793146
-4.288197278934098
-4.29837517508124
-4.308059763175947
-4.317213165766712
-4.325811876895941
-4.333846451807723
-4.341320327207679
-4.34824794829097
-4.354652450667704


# Check

In [32]:
from scipy import sparse


In [28]:
import numpy as np

def sigmaz_k(k, N) -> sparse.csr_matrix:
    res = 1
    for i in range(N):
        if i == k:
            res = sparse.kron(
                res,
                sparse.csr_matrix(
                    np.array([[1, 0], [0, -1]], dtype=np.complex)
                )
            )
        else:
            res = sparse.kron(res, sparse.eye(2, dtype=np.complex))

    return res

def sigmax_k(k, N) -> sparse.csr_matrix:
    res = 1
    for i in range(N):
        if i == k:
            res = sparse.kron(
                res,
                sparse.csr_matrix(
                    np.array([[0, 1], [1, 0]], dtype=np.complex)
                )
            )
        else:
            res = sparse.kron(res, sparse.eye(2, dtype=np.complex))

    return res

def tfi_op(N, j, h) -> sparse.csr_matrix:
    res = sparse.csr_matrix((2 ** N, 2 ** N), dtype=np.complex)

    for i in range(N - 1):
        res += -j * sigmaz_k(i, N) * sigmaz_k(i + 1, N)
        res += h * sigmax_k(i, N)

    res += h * sigmax_k(N - 1, N)

    return res

In [34]:
H_np = tfi_op(5, 1, 0.5)
exact_sol = sparse.linalg.eigs(
    sparse.csc_matrix(H_np), k=1, which="SR", return_eigenvectors=False
)[0]
print(f"Exact solution: {np.real(exact_sol):.4f}")

Exact solution: -4.4695


In [45]:
(H_np.H == H_np).min()

True